#**Assignment 3**
##Задание 1

In [ ]:
%%writefile elem_scale.cu
#include <iostream>            // Для вывода информации в консоль (cout)
#include <vector>              // Контейнер vector для хранения массива на CPU
#include <random>              // Генерация случайных чисел
#include <cuda_runtime.h>      // CUDA runtime API и CUDA events

using namespace std;

// Макрос: проверка ошибок CUDA-вызовов (если ошибка - печатаем и завершаем программу)
#define CUDA_CHECK(x) do { \
  cudaError_t e = (x); \
  if (e != cudaSuccess) { \
    cout << "CUDA error: " << cudaGetErrorString(e) \
         << " at " << __FILE__ << ":" << __LINE__ << "\n"; \
    exit(1); \
  } \
} while(0)


// Версия 1: работа только с глобальной памятью
// Каждый поток напрямую работает со своим элементом в глобальной памяти:
// читает его, умножает на коэффициент и записывает результат обратно

__global__ void scale_global(float* d, int N, float k) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;  // Вычисляем глобальный индекс элемента массива
    if (idx < N) {                                    // Проверяем, что индекс не выходит за границы массива
        d[idx] = d[idx] * k;                          // Умножаем элемент на k и записываем результат обратно
    }
}


// Версия 2: использование shared memory
// Каждый блок сначала загружает свою часть данных из глобальной памяти в shared memory,
// затем выполняет вычисления в shared memory и только потом записывает результат обратно

__global__ void scale_shared(float* d, int N, float k) {
    extern __shared__ float s[];
    // Динамическая shared memory
    // Размер задаётся при запуске kernel (block.x * sizeof(float))
    // Эта память общая для потоков одного блока и быстрее, чем global memory

    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    // Глобальный индекс элемента:
    // номер блока * размер блока + номер потока в блоке

    int tid = threadIdx.x;
    // Локальный индекс потока внутри блока
    // Используется как индекс для массива shared memory

    if (gid < N) {
        s[tid] = d[gid];
        // Каждый поток копирует один элемент из глобальной памяти в shared memory
    }
    __syncthreads();
    // Синхронизируем потоки, чтобы все данные были загружены перед вычислениями

    if (gid < N) {
        s[tid] = s[tid] * k;
        // Выполняем умножение в shared memory
    }
    __syncthreads();
    // Ждём, пока все потоки завершат вычисления перед записью обратно

    if (gid < N) {
        d[gid] = s[tid];
        // Записываем результат обратно в глобальную память
    }
}


// Функция: замер времени kernel через CUDA events

template <typename Kernel>
float timeKernel(Kernel kernel, float* d, int N, float k, dim3 grid, dim3 block, size_t shmemBytes) {
    cudaEvent_t e1, e2;                               // CUDA события: начало и конец измерения

    CUDA_CHECK(cudaEventCreate(&e1));                 // Создаём событие начала
    CUDA_CHECK(cudaEventCreate(&e2));                 // Создаём событие конца

    CUDA_CHECK(cudaEventRecord(e1));                  // Записываем событие начала
    kernel<<<grid, block, shmemBytes>>>(d, N, k);     // Запускаем kernel на GPU
    CUDA_CHECK(cudaGetLastError());                   // Проверяем, что запуск прошёл без ошибок
    CUDA_CHECK(cudaEventRecord(e2));                  // Записываем событие конца

    CUDA_CHECK(cudaEventSynchronize(e2));             // Ждём, пока kernel полностью завершится
    float ms = 0.0f;                                  // Переменная для хранения времени в миллисекундах
    CUDA_CHECK(cudaEventElapsedTime(&ms, e1, e2));    // Вычисляем время между событиями

    CUDA_CHECK(cudaEventDestroy(e1));                 // Удаляем событие начала
    CUDA_CHECK(cudaEventDestroy(e2));                 // Удаляем событие конца

    return ms;                                        // Возвращаем измеренное время выполнения kernel
}


int main() {
    const int N = 1'000'000;                          // Размер массива по заданию
    const float k = 3.5f;                             // Множитель для поэлементного умножения

    // Генерируем массив случайных чисел на CPU
    vector<float> h(N);
    mt19937 gen(42);                                  // Генератор с фиксированным seed для воспроизводимости
    uniform_real_distribution<float> dist(0.0f, 100.0f);
    for (int i = 0; i < N; ++i) h[i] = dist(gen);

    // Выделяем память на GPU
    float* d = nullptr;
    CUDA_CHECK(cudaMalloc(&d, N * sizeof(float)));

    // Копируем данные с CPU на GPU
    CUDA_CHECK(cudaMemcpy(d, h.data(), N * sizeof(float), cudaMemcpyHostToDevice));

    // Настраиваем сетку и блоки
    dim3 block(256);                                  // 256 потоков в одном блоке
    dim3 grid((N + block.x - 1) / block.x);           // Столько блоков, чтобы покрыть весь массив

    // Тест 1: версия с глобальной памятью
    CUDA_CHECK(cudaMemcpy(d, h.data(), N * sizeof(float), cudaMemcpyHostToDevice));
    float t_global = timeKernel(scale_global, d, N, k, grid, block, 0);

    vector<float> h_global(N);
    CUDA_CHECK(cudaMemcpy(h_global.data(), d, N * sizeof(float), cudaMemcpyDeviceToHost));

    // Тест 2: версия с shared memory
    CUDA_CHECK(cudaMemcpy(d, h.data(), N * sizeof(float), cudaMemcpyHostToDevice));
    size_t shmemBytes = block.x * sizeof(float);      // Размер shared memory на блок
    float t_shared = timeKernel(scale_shared, d, N, k, grid, block, shmemBytes);

    vector<float> h_shared(N);
    CUDA_CHECK(cudaMemcpy(h_shared.data(), d, N * sizeof(float), cudaMemcpyDeviceToHost));

    // Вывод результатов
    cout << "N = " << N << "\n";
    cout << "Global memory kernel time: " << t_global << " ms\n";
    cout << "Shared memory kernel time: " << t_shared << " ms\n";

    if (t_shared > 0.0f) {
        cout << "Speedup (Global/Shared): " << (t_global / t_shared) << "x\n";
    }

    CUDA_CHECK(cudaFree(d));                          // Освобождаем память на GPU
    return 0;                                         // Завершаем программу
}


Overwriting elem_scale.cu


In [ ]:
!nvcc -O3 -arch=sm_75 elem_scale.cu -o elem_scale
!./elem_scale

N = 1000000
Global memory kernel time: 0.119808 ms
Shared memory kernel time: 0.047232 ms
Speedup (Global/Shared): 2.53659x


##**Вывод**
В данном задании была реализована поэлементная обработка массива на GPU в двух вариантах: с использованием только глобальной памяти и с использованием разделяемой памяти (shared memory). Эксперименты проводились для массива размером 1 000 000 элементов.

Время выполнения версии с глобальной памятью составило 0.119808 мс, а версии с разделяемой памятью 0.047232 мс. Это соответствует ускорению примерно в 2.54 раза в пользу реализации с shared memory.

Полученный выигрыш объясняется тем, что доступ к разделяемой памяти на GPU значительно быстрее, чем к глобальной памяти. В реализации с shared memory данные сначала загружаются в быструю память блока, и дальнейшие операции выполняются уже над ними, что снижает задержки доступа и повышает общую производительность.

Таким образом, использование разделяемой памяти позволило заметно ускорить выполнение даже простой поэлементной операции при большом размере массива, что подтверждает её важную роль в оптимизации CUDA-программ.

##Задание 2

In [ ]:
%%writefile vec_add_blocks.cu
#include <iostream>            // Для вывода информации в консоль (cout)
#include <vector>              // Контейнер vector для хранения массива на CPU
#include <random>              // Генерация случайных чисел
#include <cuda_runtime.h>      // CUDA runtime API и CUDA events
#include <iomanip>             // Для красивого вывода таблиц


using namespace std;

// Макрос: проверка ошибок CUDA-вызовов (если ошибка - печатаем и завершаем программу)
#define CUDA_CHECK(x) do { \
  cudaError_t e = (x); \
  if (e != cudaSuccess) { \
    cout << "CUDA error: " << cudaGetErrorString(e) \
         << " at " << __FILE__ << ":" << __LINE__ << "\n"; \
    exit(1); \
  } \
} while(0)

// CUDA kernel: поэлементное сложение двух массивов
// Каждый поток отвечает ровно за один элемент результата
__global__ void vecAdd(const float* A, const float* B, float* C, int N) {
    // Вычисляем глобальный индекс элемента, который обрабатывает этот поток
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверяем, чтобы не выйти за границы массива (на последнем блоке могут быть лишние потоки)
    if (i < N) {
        // Основная операция: сложение соответствующих элементов массивов
        C[i] = A[i] + B[i];
    }
}

// Функция для замера времени выполнения kernel (только kernel, без memcpy)
float timeKernelVecAdd(const float* dA, const float* dB, float* dC, int N, int blockSize) {
    dim3 block(blockSize);                          // задаём размер блока потоков
    dim3 grid((N + block.x - 1) / block.x);         // считаем сколько блоков нужно под N

    cudaEvent_t e1, e2;                             // события для замера времени на GPU
    CUDA_CHECK(cudaEventCreate(&e1));
    CUDA_CHECK(cudaEventCreate(&e2));

    CUDA_CHECK(cudaEventRecord(e1));                // старт таймера на GPU
    vecAdd<<<grid, block>>>(dA, dB, dC, N);         // запуск kernel
    CUDA_CHECK(cudaGetLastError());                 // проверка, что kernel запустился нормально
    CUDA_CHECK(cudaEventRecord(e2));                // стоп таймера
    CUDA_CHECK(cudaEventSynchronize(e2));           // ждём завершения kernel

    float ms = 0.0f;                                // сюда запишется время в миллисекундах
    CUDA_CHECK(cudaEventElapsedTime(&ms, e1, e2));  // считаем разницу между событиями

    CUDA_CHECK(cudaEventDestroy(e1));
    CUDA_CHECK(cudaEventDestroy(e2));

    return ms;                                      // возвращаем время kernel в ms
}

// Быстрая проверка корректности результата
// Мы не проверяем весь массив, а смотрим только несколько случайных позиций
bool quickCheck(const vector<float>& A, const vector<float>& B, const vector<float>& C) {

    // Выбираем несколько индексов для проверки
    int idxs[] = {0, 1, 2, 123, 999, 500000, 999999};

    // Проходим по выбранным индексам
    for (int idx : idxs) {

        // Если вдруг индекс больше размера массива, просто пропускаем его
        if (idx >= (int)A.size()) continue;

        // Считаем, каким должен быть результат
        float expected = A[idx] + B[idx];

        // Берём фактическое значение из GPU-результата
        float got = C[idx];

        // Если разница слишком большая — считаем, что ошибка
        // Допускаем маленькую погрешность из-за float
        if (fabs(expected - got) > 1e-3f) return false;
    }

    // Если все проверенные элементы совпали - считаем, что всё корректно
    return true;
}

int main() {

    const int N = 1'000'000;                         // Размер массивов (по заданию)
    vector<int> blocks = {128, 256, 512};            // Три разных размера блока для эксперимента

    // Создаём массивы на CPU
    vector<float> hA(N), hB(N), hC(N);

    // Генератор случайных чисел с фиксированным seed (чтобы результаты были воспроизводимыми)
    mt19937 gen(42);
    uniform_real_distribution<float> dist(0.0f, 100.0f);

    // Заполняем массивы случайными числами
    for (int i = 0; i < N; ++i) {
        hA[i] = dist(gen);
        hB[i] = dist(gen);
    }

    // Указатели на память на GPU
    float *dA = nullptr, *dB = nullptr, *dC = nullptr;

    // Выделяем память на GPU
    CUDA_CHECK(cudaMalloc(&dA, N * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&dB, N * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&dC, N * sizeof(float)));

    // Копируем A и B на GPU (это одинаково для всех тестов)
    CUDA_CHECK(cudaMemcpy(dA, hA.data(), N * sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(dB, hB.data(), N * sizeof(float), cudaMemcpyHostToDevice));

    cout << "\nVector addition on GPU\n";
    cout << "Array size N = " << N << "\n\n";

    cout << left << setw(12) << "Block size"
     << setw(18) << "Kernel time (ms)"
     << "\n";

    // Переменные для поиска лучшего результата
    float bestTime = 1e9f;
    int bestBlock = -1;

    // Перебираем разные размеры блока
    for (int bs : blocks) {

        // Обнуляем массив результата на GPU перед каждым запуском
        CUDA_CHECK(cudaMemset(dC, 0, N * sizeof(float)));

        // Замеряем время работы kernel для текущего block size
        float ms = timeKernelVecAdd(dA, dB, dC, N, bs);

        // Копируем результат обратно на CPU для проверки
        CUDA_CHECK(cudaMemcpy(hC.data(), dC, N * sizeof(float), cudaMemcpyDeviceToHost));

        // Проверяем, правильно ли выполнено сложение
        bool ok = quickCheck(hA, hB, hC);

        // Если это лучшее время - запоминаем его
        if (ms < bestTime) {
            bestTime = ms;
            bestBlock = bs;
        }

        // Печатаем результат в таблицу
        cout << left << setw(12) << bs
             << setw(18) << fixed << setprecision(6) << ms
             << "\n";
    }

    // Печатаем лучший размер блока
    cout << "\nBest block size: " << bestBlock
         << " with time " << fixed << setprecision(6) << bestTime << " ms\n";

    // Освобождаем память на GPU
    CUDA_CHECK(cudaFree(dA));
    CUDA_CHECK(cudaFree(dB));
    CUDA_CHECK(cudaFree(dC));

    return 0;
}

Overwriting vec_add_blocks.cu


In [ ]:
!nvcc -O3 -arch=sm_75 vec_add_blocks.cu -o vec_add_blocks
!./vec_add_blocks


Vector addition on GPU
Array size N = 1000000

Block size  Kernel time (ms)  
128         0.143232          
256         0.051776          
512         0.053184          

Best block size: 256 with time 0.051776 ms


##**Вывод**

В данном задании была реализована CUDA-программа для поэлементного сложения двух массивов и исследовано влияние размера блока потоков на производительность. Эксперименты проводились для массива размером 1 000 000 элементов при размерах блока 128, 256 и 512 потоков.

Результаты показали, что наилучшее время выполнения было достигнуто при размере блока 256 потоков. При меньшем размере блока GPU используется не полностью, так как создаётся недостаточно потоков для эффективного скрытия задержек доступа к памяти. При большем размере блока возрастает конкуренция за аппаратные ресурсы, такие как регистры и разделяемая память, что снижает эффективность выполнения.

Таким образом, оптимальный размер блока потоков является компромиссом между степенью параллелизма и затратами на использование ресурсов GPU. Полученные результаты подтверждают, что правильный выбор параметров запуска CUDA-ядра существенно влияет на производительность вычислений.

##Задание 3

In [5]:
%%writefile coalescing.cu
#include <iostream>            // Для вывода информации в консоль (cout)
#include <vector>              // Контейнер vector для хранения массива на CPU
#include <random>              // Генерация случайных чисел
#include <cuda_runtime.h>      // CUDA runtime API и CUDA events
#include <iomanip>             // Для красивого вывода таблиц

using namespace std;

// Макрос: проверка ошибок CUDA-вызовов (если ошибка - печатаем и завершаем программу)
#define CUDA_CHECK(x) do { \
  cudaError_t e = (x); \
  if (e != cudaSuccess) { \
    cout << "CUDA error: " << cudaGetErrorString(e) \
         << " at " << __FILE__ << ":" << __LINE__ << "\n"; \
    exit(1); \
  } \
} while(0)

// Коалесцированный доступ: поток i берет элемент i (соседи читают соседей)
__global__ void coalescedKernel(const float* in, float* out, int N) {

// Вычисляем глобальный индекс потока
    int i = blockIdx.x * blockDim.x + threadIdx.x;

// Проверяем, чтобы не выйти за границы массива
    if (i < N) {
        // Поток читает in[i] и записывает в out[i]
        // Соседние потоки читают соседние адреса -> коалесцированный доступ
        out[i] = in[i] * 2.0f;
}

// Некоалесцированный доступ: поток i берет элемент j (скачки по памяти), потоки читают данные "вразнобой"

__global__ void nonCoalescedKernel(const float* in, float* out, int N, int STRIDE) {

// Глобальный индекс потока
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < N) {

        // Вычисляем "разбросанный" индекс
        // (i * STRIDE) % N гарантирует, что мы останемся внутри массива
        int j = (i * STRIDE) % N;

        // Каждый поток читает и пишет в "случайное" место в памяти
        // Это ломает коалесцинг и делает доступ медленным
        out[j] = in[j] * 2.0f;
    }
}

// Функция измерения времени: коалесцированный kernel

float timeKernelCoalesced(const float* dIn, float* dOut, int N,
                          dim3 grid, dim3 block, int repeats) {

    // Создаём два CUDA-события: начало и конец
    cudaEvent_t e1, e2;
    CUDA_CHECK(cudaEventCreate(&e1));
    CUDA_CHECK(cudaEventCreate(&e2));

    // Фиксируем момент старта
    CUDA_CHECK(cudaEventRecord(e1));

    // Запускаем kernel много раз (repeats), чтобы получить стабильный замер
    for (int r = 0; r < repeats; ++r) {
        coalescedKernel<<<grid, block>>>(dIn, dOut, N);
    }

    // Проверяем, что kernel запустился без ошибок
    CUDA_CHECK(cudaGetLastError());

    // Фиксируем момент окончания
    CUDA_CHECK(cudaEventRecord(e2));

    // Ждём, пока всё выполнится
    CUDA_CHECK(cudaEventSynchronize(e2));

    // Переменная для времени (в миллисекундах)
    float ms = 0.0f;

    // Считаем разницу между e1 и e2
    CUDA_CHECK(cudaEventElapsedTime(&ms, e1, e2));

    // Удаляем события
    CUDA_CHECK(cudaEventDestroy(e1));
    CUDA_CHECK(cudaEventDestroy(e2));

    // Возвращаем время выполнения
    return ms;
}


// Функция измерения времени: некоалесцированный Kernel

float timeKernelNonCoalesced(const float* dIn, float* dOut, int N,
                             dim3 grid, dim3 block, int repeats, int stride) {

    cudaEvent_t e1, e2;
    CUDA_CHECK(cudaEventCreate(&e1));
    CUDA_CHECK(cudaEventCreate(&e2));

    CUDA_CHECK(cudaEventRecord(e1));

    // Запускаем kernel много раз для стабильности
    for (int r = 0; r < repeats; ++r) {
        nonCoalescedKernel<<<grid, block>>>(dIn, dOut, N, stride);
    }

    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaEventRecord(e2));
    CUDA_CHECK(cudaEventSynchronize(e2));

    float ms = 0.0f;
    CUDA_CHECK(cudaEventElapsedTime(&ms, e1, e2));

    CUDA_CHECK(cudaEventDestroy(e1));
    CUDA_CHECK(cudaEventDestroy(e2));

    return ms;
}

int main() {

    const int N = 1'000'000;          // размер массива по заданию
    const int BLOCK = 256;            // стандартный удобный размер блока
    const int REPEATS = 500;          // повторяем kernel много раз для точности
    const int STRIDE = 997;           // шаг доступа (некратный 32), чтобы сломать коалесцинг

    // Создаём массив на CPU
    vector<float> h(N);

    // Генератор случайных чисел
    mt19937 gen(42);
    uniform_real_distribution<float> dist(0.0f, 100.0f);

    // Заполняем массив случайными числами
    for (int i = 0; i < N; ++i) {
        h[i] = dist(gen);
    }

    // Указатели на память на GPU
    float *dIn = nullptr, *dOut = nullptr;

    // Выделяем память на GPU
    CUDA_CHECK(cudaMalloc(&dIn, N * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&dOut, N * sizeof(float)));

    // Копируем данные с CPU на GPU
    CUDA_CHECK(cudaMemcpy(dIn, h.data(), N * sizeof(float), cudaMemcpyHostToDevice));

    // Настройки запуска kernel
    dim3 block(BLOCK);                       // сколько потоков в блоке
    dim3 grid((N + BLOCK - 1) / BLOCK);     // сколько блоков нужно, чтобы покрыть N


    // Первый запуск иногда медленнее из-за инициализации,
    // поэтому мы его не учитываем в замере

    coalescedKernel<<<grid, block>>>(dIn, dOut, N);
    nonCoalescedKernel<<<grid, block>>>(dIn, dOut, N, STRIDE);
    CUDA_CHECK(cudaDeviceSynchronize());

    // Реальный замер времени
    float tCoalesced = timeKernelCoalesced(dIn, dOut, N, grid, block, REPEATS);
    float tNon       = timeKernelNonCoalesced(dIn, dOut, N, grid, block, REPEATS, STRIDE);

    // Результаты

    cout << "Array size N = " << N << "\n";
    cout << "Block size = " << BLOCK << "\n";
    cout << "Repeats = " << REPEATS << "\n";
    cout << "Stride (non-coalesced) = " << STRIDE << "\n\n";

    cout << fixed << setprecision(6);
    cout << "Coalesced time (ms):     " << tCoalesced << "\n";
    cout << "Non-coalesced time (ms): " << tNon << "\n";

    if (tCoalesced > 0.0f) {
        cout << "Slowdown (Non/Coalesced): " << (tNon / tCoalesced) << "x\n";
    }

    // Освобождаем память на GPU
    CUDA_CHECK(cudaFree(dIn));
    CUDA_CHECK(cudaFree(dOut));

    return 0;
}

Overwriting coalescing.cu


In [6]:
!nvcc -O3 -arch=sm_75 coalescing.cu -o coalescing
!./coalescing

Array size N = 1000000
Block size = 256
Repeats = 500
Stride (non-coalesced) = 997

Coalesced time (ms):     19.030016
Non-coalesced time (ms): 320.446289
Slowdown (Non/Coalesced): 16.838993x


##**Вывод**

В работе была реализована обработка массива на GPU с двумя вариантами доступа к глобальной памяти: коалесцированным и некоалесцированным. Эксперимент проведён для массива размером 1 000 000 элементов при размере блока 256 потоков, время измерялось по 500 повторениям для повышения стабильности результатов.

Результаты показали, что коалесцированный доступ выполняется значительно быстрее: 19.03 ms против 320.45 ms для некоалесцированного доступа. Некоалесцированный вариант оказался медленнее примерно в 16.84 раза. Это объясняется тем, что при коалесцированном доступе соседние потоки обращаются к соседним адресам памяти, и GPU может объединять запросы в меньшее число транзакций. При некоалесцированном доступе потоки обращаются к памяти с большим шагом, что приводит к большему числу отдельных обращений и снижает эффективность использования пропускной способности памяти.

Таким образом, эксперимент подтверждает, что правильная организация доступа к глобальной памяти (коалесцирование) является критически важной для производительности CUDA-программ.

##Задание 4

In [5]:
%%writefile task4.cu
#include <iostream>              // вывод в консоль
#include <vector>                // массивы на CPU
#include <random>                // генерация случайных чисел
#include <cuda_runtime.h>        // CUDA runtime API (cudaMalloc, cudaMemcpy, events)
#include <iomanip>               // красивое форматирование таблицы (setw, setprecision)

using namespace std;

// Макрос для проверки ошибок CUDA (если ошибка, печатаем и выходим)
#define CUDA_CHECK(x) do { \
  cudaError_t e = (x); \
  if (e != cudaSuccess) { \
    cout << "CUDA error: " << cudaGetErrorString(e) \
         << " at " << __FILE__ << ":" << __LINE__ << "\n"; \
    exit(1); \
  } \
} while(0)


// Берём программу из задания 2 (поэлементное сложение массивов)
// и подбираем оптимальный размер блока (block size), а grid size считаем автоматически.

// CUDA kernel: поэлементное сложение двух массивов: C[i] = A[i] + B[i]
__global__ void vecAdd(const float* A, const float* B, float* C, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;   // глобальный индекс элемента для данного потока
    if (i < N) {                                     // защита от выхода за границы массива
        C[i] = A[i] + B[i];                          // основная операция: сложение
    }
}


// Функция для замера времени выполнения kernel через CUDA events
// Важно: делаем несколько повторов и возвращаем среднее время одного запуска
float timeKernelVecAdd(const float* dA, const float* dB, float* dC, int N,
                       int blockSize, int repeats) {

    dim3 block(blockSize);                           // block: сколько потоков в одном блоке
    dim3 grid((N + block.x - 1) / block.x);          // grid: сколько блоков нужно, чтобы покрыть N элементов

    // Прогрев, чтобы первый запуск не дал "лишнее" время
    vecAdd<<<grid, block>>>(dA, dB, dC, N);
    CUDA_CHECK(cudaDeviceSynchronize());

    // CUDA events для измерения времени на GPU
    cudaEvent_t e1, e2;
    CUDA_CHECK(cudaEventCreate(&e1));
    CUDA_CHECK(cudaEventCreate(&e2));

    CUDA_CHECK(cudaEventRecord(e1));                 // отметили старт на GPU таймлайне

    for (int r = 0; r < repeats; ++r) {              // повторяем kernel несколько раз для устойчивости
        vecAdd<<<grid, block>>>(dA, dB, dC, N);      // запуск kernel
    }

    CUDA_CHECK(cudaGetLastError());                  // проверка: kernel запустился без ошибок
    CUDA_CHECK(cudaEventRecord(e2));                 // отметили конец
    CUDA_CHECK(cudaEventSynchronize(e2));            // ждём завершения всех запусков kernel

    float totalMs = 0.0f;                            // суммарное время за все repeats
    CUDA_CHECK(cudaEventElapsedTime(&totalMs, e1, e2));

    CUDA_CHECK(cudaEventDestroy(e1));                // чистим события
    CUDA_CHECK(cudaEventDestroy(e2));

    return totalMs / repeats;                        // возвращаем среднее время одного запуска
}


// Утилита: печать одной строки таблицы
void printRow(int blockSize, int gridX, double ms) {
    cout << left << setw(12) << blockSize            // размер блока
         << setw(16) << gridX                        // grid.x (количество блоков)
         << fixed << setprecision(6) << ms << "\n";  // время (мс)
}


int main() {
    // Размер массива
    const int N = 1'000'000;                         // число элементов
    const int repeats = 300;                         // сколько раз повторяем kernel для замера

    // Генерация входных данных на CPU
    vector<float> hA(N), hB(N);                      // два входных массива
    mt19937 gen(42);                                 // фиксированный seed для повторяемости результатов
    uniform_real_distribution<float> dist(0.0f, 1.0f);// случайные числа [0;1]

    for (int i = 0; i < N; ++i) {                    // заполняем массивы случайными значениями
        hA[i] = dist(gen);
        hB[i] = dist(gen);
    }

    // Выделяем память на GPU
    float *dA = nullptr, *dB = nullptr, *dC = nullptr;
    CUDA_CHECK(cudaMalloc(&dA, N * sizeof(float)));  // GPU память под A
    CUDA_CHECK(cudaMalloc(&dB, N * sizeof(float)));  // GPU память под B
    CUDA_CHECK(cudaMalloc(&dC, N * sizeof(float)));  // GPU память под C (результат)

    // Копируем данные на GPU
    CUDA_CHECK(cudaMemcpy(dA, hA.data(), N * sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(dB, hB.data(), N * sizeof(float), cudaMemcpyHostToDevice));

    // Кандидаты block size (типичные значения, кратные warp=32)
    vector<int> candidates = {64, 128, 256, 512, 1024};

    cout << "Vector addition tuning (Task 4)\n";
    cout << "Array size N = " << N << "\n\n";

    // Подбор оптимальной конфигурации: ищем лучший block size по времени
    cout << left << setw(12) << "Block size"
         << setw(16) << "Grid.x"
         << "Kernel time (ms)\n";

    double bestTime = 1e100;                         // лучшее время
    int bestBlock = -1;                              // лучший размер блока
    int bestGridX = -1;                              // сколько блоков по x у лучшей конфигурации

    for (int bs : candidates) {                      // пробуем каждый block size
        dim3 block(bs);                              // создаём block
        dim3 grid((N + block.x - 1) / block.x);      // считаем grid.x

        double t = timeKernelVecAdd(dA, dB, dC, N, bs, repeats);  // замеряем время

        printRow(bs, grid.x, t);                     // печатаем строку в таблице

        if (t < bestTime) {                          // обновляем лучший вариант
            bestTime = t;
            bestBlock = bs;
            bestGridX = grid.x;
        }
    }

    // Сравнение "неоптимальной" и "оптимальной" конфигурации
    // В качестве неоптимальной обычно берут слишком маленький block size
    int badBlock = 64;                               // пример неоптимального (маленький блок)
    dim3 badGrid((N + badBlock - 1) / badBlock);     // grid для badBlock
    double badTime = timeKernelVecAdd(dA, dB, dC, N, badBlock, repeats);

    cout << "\nComparison (Task 4)\n";
    cout << "Non-optimal config: block = " << badBlock
         << ", grid.x = " << badGrid.x
         << ", time = " << fixed << setprecision(6) << badTime << " ms\n";

    cout << "Optimal config:     block = " << bestBlock
         << ", grid.x = " << bestGridX
         << ", time = " << fixed << setprecision(6) << bestTime << " ms\n";

    // Освобождаем память на GPU
    CUDA_CHECK(cudaFree(dA));
    CUDA_CHECK(cudaFree(dB));
    CUDA_CHECK(cudaFree(dC));

    return 0;
}


Writing task4.cu


In [4]:
!nvcc -O3 -arch=sm_75 task4.cu -o task4
!./task4

Vector addition tuning (Task 4)
Array size N = 1000000

Block size  Grid.x          Kernel time (ms)
64          15625           0.050439
128         7813            0.049268
256         3907            0.049282
512         1954            0.049417
1024        977             0.051004

Comparison (Task 4)
Non-optimal config: block = 64, grid.x = 15625, time = 0.050438 ms
Optimal config:     block = 128, grid.x = 7813, time = 0.049268 ms


##**Вывод**
В качестве базовой программы для подбора оптимальной конфигурации было выбрано Задание 2, поэлементное сложение двух массивов.  Эксперимент проводился для массива размером 1 000 000 элементов, при этом выполнялся перебор нескольких значений размера блока потоков (64, 128, 256, 512, 1024), а размер сетки рассчитывался автоматически так, чтобы покрыть весь массив.

По результатам измерений наименьшее время выполнения показал блок размером 128 потоков (0.049268 мс), что было принято в качестве оптимальной конфигурации. В качестве неоптимальной конфигурации был выбран меньший размер блока 64 потока, при котором время выполнения составило 0.050438 мс.

Сравнение показало, что оптимизированная конфигурация (block = 128, grid.x = 7813) обеспечивает немного более высокую производительность по сравнению с неоптимальной (block = 64, grid.x = 15625). Это объясняется более эффективной загрузкой GPU и лучшим балансом между количеством потоков в блоке и числом блоков в сетке.

Таким образом, эксперимент подтвердил, что выбор размера блока потоков влияет на производительность CUDA-программы, и подбор оптимальной конфигурации позволяет улучшить время выполнения.